####  1. Load API Key and Required Libraries

This code initializes the environment by:
- Loading environment variables from a .env file
- Retrieving the Gemini API key
- Setting up the API endpoint URL for making requests


In [6]:
import os
import requests
from dotenv import load_dotenv

load_dotenv()
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
if GEMINI_API_KEY:
    print("Gemini API key loaded successfully.")
else:
    print("Failed to load Gemini API key.")
API_URL = f"https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent?key={GEMINI_API_KEY}"

Gemini API key loaded successfully.


#### 2. Load and Split Lecture PDF

Processes PDF documents by:
- Loading PDF files using PyMuPDFLoader
- Splitting content into smaller chunks for processing
- Creating a context string with character limits (~12k)

In [7]:
import glob
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Get all PDF files in the folder
pdf_files = glob.glob("docs/*.pdf")

# Initialize empty lists to store all docs and chunks
all_docs = []
all_chunks = []

# Process each PDF file
for pdf_file in pdf_files:
    loader = PyMuPDFLoader(pdf_file)
    docs = loader.load()
    all_docs.extend(docs)
    print(f"Successfully loaded {len(docs)} pages from {pdf_file}")

# Split all documents into chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
chunks = splitter.split_documents(all_docs)
print(f"Documents split into {len(chunks)} chunks")

# Combine into a single context string (limit to ~12k chars)
context = "\n\n".join([doc.page_content for doc in chunks])[:12000]
print(f"Context string created with {len(context)} characters")


Successfully loaded 28 pages from PDF
Document split into 28 chunks
Context string created with 7743 characters


#### 3. Ask Questions from Gemini

Enables Q&A functionality through:
- An ask_gemini() function that takes questions
- Uses the processed PDF content as context
- Returns AI-generated answers via the Gemini API

In [8]:
def ask_gemini(question):
    prompt = f"""
Use the following MADD lecture notes to answer the question.
Please include page numbers from the lecture notes where you found the information.

Lecture Notes:
{context}

Question:
{question}

Please format your answer to include relevant page numbers in brackets [Page X] where the information comes from.
"""
    payload = {
        "contents": [
            {
                "parts": [{"text": prompt}]
            }
        ]
    }
    headers = {"Content-Type": "application/json"}
    
    response = requests.post(API_URL, headers=headers, json=payload)
    
    if response.status_code == 200:
        return response.json()["candidates"][0]["content"]["parts"][0]["text"]
    else:
        return f"Error: {response.status_code} - {response.text}"


#### 4. Print the Answer

In [9]:
print(ask_gemini("Explain the closure in swift?"))


In Swift, a closure is essentially a self-contained block of functionality that can be passed around and used in your code. Swift allows you to use a function as a type, just like an integer or a string. [Page 18].

Here's a breakdown:

*   **Functions as Types:** Swift treats functions as first-class citizens. This means you can assign a function to a variable. [Page 18].

*   **Capture Values:** Closures can "capture" references to constants and variables from the surrounding context (the scope in which they are defined). This is known as "closing over" values. [Page 18]. This means the closure can access and even modify those variables, even if the surrounding scope is no longer active.

*   **Use Cases:** Closures are particularly useful in scenarios where you want to store some functionality to be executed later or repeatedly. Some common use cases include:
    *   Waiting for a period of time before executing code. [Page 19].
    *   Handling UI animations or interactions that ne